In [2]:
#pip install vaderSentiment
from bs4 import BeautifulSoup as bs
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import re
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from statistics import mean
executable_path = {'executable_path': ChromeDriverManager().install()}



====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Get LATEST driver version for 92.0.4515
Trying to download new driver from https://chromedriver.storage.googleapis.com/92.0.4515.107/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\Rajan\.wdm\drivers\chromedriver\win32\92.0.4515.107]


In [11]:
def review_finder(movie_title):
    
    formatted_title = movie_title.strip().lower()
    query = re.sub(" ", "_", formatted_title)
    base_url = "https://www.rottentomatoes.com/m/"
    full_url = base_url + query

    browser = Browser('chrome', **executable_path, headless=False)
    browser.visit(full_url)
    html = browser.html
    soup = bs(html, "html.parser")

    data = soup.find_all(class_ = "mop-audience-reviews__review--comment clamp clamp-4 js-clamp")
    review_list = []
    for review in data:
        review_list.append(review.text)
    
    scores_list=[]
    vader_scores=[]
    positive_vader_scores=[]
    for reviews in review_list:
            analyser = SentimentIntensityAnalyzer()
            scores = analyser.polarity_scores(reviews)
            compound_score = scores['compound']
            positive_score = scores['pos']
            scores_list.append(scores)
            vader_scores.append(compound_score)
            positive_vader_scores.append(positive_score)
            
    avg_vader_score = mean(vader_scores) * 100
    avg_positive_score = mean(positive_vader_scores) * 100
    
    return avg_vader_score, avg_positive_score, scores_list, review_list
    



In [12]:
x = review_finder("inception")

In [23]:
data_df = pd.read_csv("MoviesOnStreamingPlatforms_updated.csv")

In [25]:
data_df.head()

,Unnamed: 0,ID,Title,Year,Age,IMDb,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,Type,Directors,Genres,Country,Language,Runtime
0,0,1,Inception,2010,13+,8.8,87%,1,0,0,0,0,Christopher Nolan,"Action,Adventure,Sci-Fi,Thriller","United States,United Kingdom","English,Japanese,French",148.0
1,1,2,The Matrix,1999,18+,8.7,87%,1,0,0,0,0,"Lana Wachowski,Lilly Wachowski","Action,Sci-Fi",United States,English,136.0
2,2,3,Avengers: Infinity War,2018,13+,8.5,84%,1,0,0,0,0,"Anthony Russo,Joe Russo","Action,Adventure,Sci-Fi",United States,English,149.0
3,3,4,Back to the Future,1985,7+,8.5,96%,1,0,0,0,0,Robert Zemeckis,"Adventure,Comedy,Sci-Fi",United States,English,116.0
4,4,5,"The Good, the Bad and the Ugly",1966,18+,8.8,97%,1,0,1,0,0,Sergio Leone,Western,"Italy,Spain,West Germany",Italian,161.0


In [45]:
x[2]

[{'neg': 0.0, 'neu': 0.238, 'pos': 0.762, 'compound': 0.8625},
 {'neg': 0.0, 'neu': 0.565, 'pos': 0.435, 'compound': 0.9062},
 {'neg': 0.071, 'neu': 0.79, 'pos': 0.14, 'compound': 0.8828},
 {'neg': 0.0, 'neu': 0.476, 'pos': 0.524, 'compound': 0.9786}]

In [34]:
data_df["Average Vader Score"] =  "NaN"
data_df["Average Positive Score"] = "NaN"
data_df["All Scores"] = "NaN"

In [35]:
data_df.head()

,Unnamed: 0,ID,Title,Year,Age,IMDb,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,Type,Directors,Genres,Country,Language,Runtime,Average Vader Score,Average Positive Score,All Scores
0,0,1,Inception,2010,13+,8.8,87%,1,0,0,0,0,Christopher Nolan,"Action,Adventure,Sci-Fi,Thriller","United States,United Kingdom","English,Japanese,French",148.0,NaN,NaN,NaN
1,1,2,The Matrix,1999,18+,8.7,87%,1,0,0,0,0,"Lana Wachowski,Lilly Wachowski","Action,Sci-Fi",United States,English,136.0,NaN,NaN,NaN
2,2,3,Avengers: Infinity War,2018,13+,8.5,84%,1,0,0,0,0,"Anthony Russo,Joe Russo","Action,Adventure,Sci-Fi",United States,English,149.0,NaN,NaN,NaN
3,3,4,Back to the Future,1985,7+,8.5,96%,1,0,0,0,0,Robert Zemeckis,"Adventure,Comedy,Sci-Fi",United States,English,116.0,NaN,NaN,NaN
4,4,5,"The Good, the Bad and the Ugly",1966,18+,8.8,97%,1,0,1,0,0,Sergio Leone,Western,"Italy,Spain,West Germany",Italian,161.0,NaN,NaN,NaN


In [46]:
data_df["Average Vader Score"][0] = x[0]
data_df["Average Positive Score"][0] = x[1]
data_df["All Scores"][0] = x[2]

<ipython-input-46-fcdec6b9e858>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df["Average Vader Score"][0] = x[0]
<ipython-input-46-fcdec6b9e858>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df["Average Positive Score"][0] = x[1]
<ipython-input-46-fcdec6b9e858>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df["All Scores"][0] = x[2]


In [47]:
data_df.head()

,Unnamed: 0,ID,Title,Year,Age,IMDb,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,Type,Directors,Genres,Country,Language,Runtime,Average Vader Score,Average Positive Score,All Scores
0,0,1,Inception,2010,13+,8.8,87%,1,0,0,0,0,Christopher Nolan,"Action,Adventure,Sci-Fi,Thriller","United States,United Kingdom","English,Japanese,French",148.0,90.7525,46.525,"[{'neg': 0.0, 'neu': 0.238, 'pos': 0.762, 'com..."
1,1,2,The Matrix,1999,18+,8.7,87%,1,0,0,0,0,"Lana Wachowski,Lilly Wachowski","Action,Sci-Fi",United States,English,136.0,NaN,NaN,NaN
2,2,3,Avengers: Infinity War,2018,13+,8.5,84%,1,0,0,0,0,"Anthony Russo,Joe Russo","Action,Adventure,Sci-Fi",United States,English,149.0,NaN,NaN,NaN
3,3,4,Back to the Future,1985,7+,8.5,96%,1,0,0,0,0,Robert Zemeckis,"Adventure,Comedy,Sci-Fi",United States,English,116.0,NaN,NaN,NaN
4,4,5,"The Good, the Bad and the Ugly",1966,18+,8.8,97%,1,0,1,0,0,Sergio Leone,Western,"Italy,Spain,West Germany",Italian,161.0,NaN,NaN,NaN


In [63]:
data_df["Title"].head()

0                         Inception
1                        The Matrix
2            Avengers: Infinity War
3                Back to the Future
4    The Good, the Bad and the Ugly
Name: Title, dtype: object

In [146]:
x[0]

59.0425

In [186]:
x = review_finder("Dangal")

StatisticsError: mean requires at least one data point

In [184]:
data_df["Average Vader Score"   ][34] = x[0]
data_df["Average Positive Score"][34] = x[1]
data_df["All Scores"            ][34] = x[2]

<ipython-input-184-a576f630af1d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df["Average Vader Score"   ][33] = x[0]
<ipython-input-184-a576f630af1d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df["Average Positive Score"][33] = x[1]
<ipython-input-184-a576f630af1d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df["All Scores"            ][33] = x[2]


In [185]:
data_df[["Title", "Average Vader Score", "Average Positive Score"]].head(35)

,Title,Average Vader Score,Average Positive Score
0,Inception,90.7525,46.525
1,The Matrix,58.6425,14.175
2,Avengers: Infinity War,79.2775,15.775
3,Back to the Future,NaN,NaN
4,"The Good, the Bad and the Ugly",NaN,NaN
5,Spider-Man: Into the Spider-Verse,71.1125,18.825
6,The Pianist,NaN,NaN
7,Django Unchained,1.5225,14.925
8,Raiders of the Lost Ark,77.2175,16.975
9,Inglourious Basterds,-35.665,7


In [188]:
data_df.to_csv("All_Movies.csv")

In [191]:
filtered_df = data_df[data_df["Average Vader Score"] != "NaN"]

In [193]:
filtered_df.to_csv("Only_Vader_Scored.csv")

In [127]:
def critic_review_finder(movie_title):
    
    formatted_title = movie_title.strip().lower()
    query = re.sub(" ", "_", formatted_title)
    base_url = "https://www.rottentomatoes.com/m/"
    full_url = base_url + query

    browser = Browser('chrome', **executable_path, headless=False)
    browser.visit(full_url)
    html = browser.html
    soup = bs(html, "html.parser")

    data = soup.find_all(class_ = "quote_bubble__quote")
    critic_reviews = [cd.find("p").get_text().strip() for cd in data]
    
    
    scores_list=[]
    vader_scores=[]
    positive_vader_scores=[]
    for reviews in critic_reviews:
            analyser = SentimentIntensityAnalyzer()
            scores = analyser.polarity_scores(reviews)
            compound_score = scores['compound']
            positive_score = scores['pos']
            scores_list.append(scores)
            vader_scores.append(compound_score)
            positive_vader_scores.append(positive_score)
            
    avg_vader_score = mean(vader_scores) * 100
    avg_positive_score = mean(positive_vader_scores) * 100
    
    return avg_vader_score, avg_positive_score, scores_list, critic_reviews
    



In [143]:
vader_df = pd.read_csv("Only_Vader_Scored.csv")

In [145]:
vader_df["Average Vader Score (Critic)"] = None
vader_df["Average Positive Score (Critic)"] = None
lament["All scores (Critic)"] = None

In [148]:
vader_df["Title"]

0                              Inception
1                             The Matrix
2                 Avengers: Infinity War
3      Spider-Man: Into the Spider-Verse
4                       Django Unchained
5                Raiders of the Lost Ark
6                   Inglourious Basterds
7                            Taxi Driver
8                        Pan's Labyrinth
9                                   Room
10       Monty Python and the Holy Grail
11    Indiana Jones and the Last Crusade
12                         Groundhog Day
13                     The King's Speech
14                                   Her
15                   There Will Be Blood
16                    The Social Network
17                    Dallas Buyers Club
18       The Perks of Being a Wallflower
19                            Ex Machina
20               Silver Linings Playbook
21                            District 9
22                          The Irishman
23                                  Moon
24              

In [238]:
movie_data = critic_review_finder("incredibles 2")

In [239]:
vader_df["Average Vader Score (Critic)"]   [26] = movie_data[0]
vader_df["Average Positive Score (Critic)"][26] = movie_data[1]
vader_df["All scores (Critic)"]            [26] = movie_data[2]

<ipython-input-239-d6c2d96695e4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lament["Average Vader Score (Critic)"]   [26] = movie_data[0]
<ipython-input-239-d6c2d96695e4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lament["Average Positive Score (Critic)"][26] = movie_data[1]
<ipython-input-239-d6c2d96695e4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lament["All scores (Critic)"]            [26] = movie_data[2]


In [240]:
vader_df[["Title", "Average Vader Score", "Average Vader Score (Critic)"]].tail(5)

,Title,Average Vader Score,Average Vader Score (Critic)
22,The Irishman,67.9675,32.9263
23,Moon,16.5600,12.4679
24,Marriage Story,14.9800,33.3742
25,Train to Busan,14.8075,13.9383
26,Incredibles 2,84.7075,59.9125


In [242]:
vader_df.to_csv("Only_Vader_Scored_With_Critics.csv")